# Simple Rating Biases
* Computes a bias for each user and for each item
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = m + u_i + a_j$
* $m = \text{mean}_{ij}(r_{ij})$
* $u_i = \text{mean}_j(r_{ij}) - m$
* $a_j = \text{mean}_i(r_{ij}) - m$

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlpha.ipynb");

In [ ]:
version = ""
dataset = ""
medium = "";

In [ ]:
import DataFrames: combine, DataFrame, groupby
import Statistics: mean

In [ ]:
const name = "simple/$version/$dataset/$medium/rating_baseline";

In [ ]:
function get_df(dataset)
    df = as_metric(
        get_split(dataset, "train", medium, [:userid, :itemid, :rating]),
        "rating",
    )
    DataFrame(user = df.userid, item = df.itemid, rating = convert.(Float64, df.rating))
end;

In [ ]:
function train_model(dataset)
    training = get_df(dataset)
    μ = mean(training.rating)
    a = combine(groupby(training, :item), :rating => mean => :rating)
    a = Dict(Pair.(a.item, a.rating .- μ))
    a = [get(a, i, 0) for i = 1:num_items(medium)]
    write_params(Dict("a" => convert.(Float32, a), "μ" => convert(Float32, μ)), name)
end;

In [ ]:
function model(dfs, userids, medium)
    params = read_params(name)
    μ = params["μ"]
    a = params["a"]
    preds = Dict{Int32,Vector{Float32}}()
    rd = as_metric(dfs[medium], "rating")
    df = DataFrame(user = rd.userid, item = rd.itemid, rating = rd.rating)
    u = combine(groupby(df, :user), :rating => mean => :rating)
    u = Dict(Pair.(u.user, u.rating .- μ))
    @showprogress for user in userids
        preds[user] = μ + get(u, user, 0) .+ a
    end
    preds
end;

In [ ]:
train_model(dataset);

In [ ]:
write_alpha((dfs, userids) -> model(dfs, userids, medium), medium, name, [medium])
print_losses(medium, "rating", [name]);